In [1]:
!pip install tpot

    100% |████████████████████████████████| 890kB 1.5MB/s eta 0:00:01
    100% |████████████████████████████████| 942kB 1.5MB/s eta 0:00:01
  Running setup.py bdist_wheel for tpot ... done
  Stored in directory: /root/.cache/pip/wheels/d2/54/33/7549c05095a6a38d3de610f88f2d075e56617ff887dce6d54e
  Running setup.py bdist_wheel for deap ... done
  Stored in directory: /root/.cache/pip/wheels/82/aa/67/2c93e17c84646c86099fda53ee0b3329372dcf94dd8789fd13
  Running setup.py bdist_wheel for stopit ... done
  Stored in directory: /root/.cache/pip/wheels/95/fc/6b/0289a3bce1635be994845f61cbaa91a7ac93dfc453229f0442
Successfully built tpot deap stopit


In [2]:
from tpot import TPOTClassifier
from sklearn.cross_validation import train_test_split
import pandas as pd
import numpy as np

/usr/local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
# Load the data
telescope = pd.read_csv("MAGIC Gamma Telescope Data.csv");
telescope.head()

,Flength,Fwidth,Fsize,Fconc,Fconc1,Fasym,Fm3long,Fm3trans,Falpha,Fdist,Class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


In [4]:
telescope_shuffle = telescope.iloc[np.random.permutation(len(telescope))]
telescope_shuffle.head()

,Flength,Fwidth,Fsize,Fconc,Fconc1,Fasym,Fm3long,Fm3trans,Falpha,Fdist,Class
15448,123.5800,25.0260,2.8451,0.4114,0.2379,-141.3120,-99.6804,-15.4817,26.7260,371.4970,h
9034,31.6204,14.3757,2.7462,0.3605,0.1892,-11.9099,22.5139,6.0822,11.5864,199.3630,g
9180,63.4767,28.9711,3.3728,0.1318,0.0663,18.5203,37.3319,11.5376,11.8804,213.1290,g
13356,18.6083,10.2012,2.2480,0.5932,0.3192,0.9860,-6.7047,5.9809,88.3435,42.7209,h
14024,19.0819,16.9368,2.8296,0.5107,0.2909,-9.0182,-15.9421,12.1762,13.1186,276.2790,h


In [5]:
tele = telescope_shuffle.reset_index(drop=True)
tele.head()

,Flength,Fwidth,Fsize,Fconc,Fconc1,Fasym,Fm3long,Fm3trans,Falpha,Fdist,Class
0,123.5800,25.0260,2.8451,0.4114,0.2379,-141.3120,-99.6804,-15.4817,26.7260,371.4970,h
1,31.6204,14.3757,2.7462,0.3605,0.1892,-11.9099,22.5139,6.0822,11.5864,199.3630,g
2,63.4767,28.9711,3.3728,0.1318,0.0663,18.5203,37.3319,11.5376,11.8804,213.1290,g
3,18.6083,10.2012,2.2480,0.5932,0.3192,0.9860,-6.7047,5.9809,88.3435,42.7209,h
4,19.0819,16.9368,2.8296,0.5107,0.2909,-9.0182,-15.9421,12.1762,13.1186,276.2790,h


# Pre-Processing Data

In [6]:
tele.dtypes

Flength     float64
Fwidth      float64
Fsize       float64
Fconc       float64
Fconc1      float64
Fasym       float64
Fm3long     float64
Fm3trans    float64
Falpha      float64
Fdist       float64
Class        object
dtype: object

In [7]:
for cat in ["Class"]:
    print("Levels for category '{0}': {1}".format(cat, tele[cat].unique()))

Levels for category 'Class': ['h' 'g']


In [8]:
tele["Class"] = tele["Class"].map({'g':0, 'h':1})

In [9]:
tele = tele.fillna(-999)
pd.isnull(tele).any()

Flength     False
Fwidth      False
Fsize       False
Fconc       False
Fconc1      False
Fasym       False
Fm3long     False
Fm3trans    False
Falpha      False
Fdist       False
Class       False
dtype: bool

In [10]:
tele.shape

(19020, 11)

In [11]:
tele_class = tele["Class"].values

# Data Analysis using TPOT

In [12]:
training_indices, validation_indices = training_indices, test_indices = train_test_split(tele.index, stratify = tele_class, train_size = 0.75, test_size=0.25)
training_indices.size, validation_indices.size

(14265, 4755)

In [ ]:
# tpot = TPOTClassifier(verbosity = 2, population_size=15, max_eval_time_mins=0.04, max_time_mins=3)
tpot = TPOTClassifier(verbosity = 2, generations=5, population_size=20)
tpot.fit(tele.drop("Class", axis=1).loc[training_indices].values, tele.loc[training_indices, "Class"].values)

Optimization Progress:  33%|███▎      | 40/120 [04:25<03:06,  2.33s/pipeline]

Generation 1 - Current best internal CV score: 0.8812474102933046


Optimization Progress:  50%|█████     | 60/120 [07:51<08:00,  8.00s/pipeline]

Generation 2 - Current best internal CV score: 0.8812474102933046


Optimization Progress:  67%|██████▋   | 80/120 [15:43<07:12, 10.80s/pipeline]

Generation 3 - Current best internal CV score: 0.8828600923179535


Optimization Progress:  83%|████████▎ | 100/120 [28:00<10:50, 32.54s/pipeline]

Generation 4 - Current best internal CV score: 0.8828600923179535


Generation 5 - Current best internal CV score: 0.8828600923179535

Best pipeline: XGBClassifier(input_matrix, learning_rate=0.1, max_depth=7, min_child_weight=1, n_estimators=100, nthread=1, subsample=0.9)


TPOTClassifier(config_dict={'sklearn.preprocessing.MaxAbsScaler': {}, 'sklearn.feature_selection.SelectFwe': {'score_func': {'sklearn.feature_selection.f_classif': None}, 'alpha': array([ 0.   ,  0.001,  0.002,  0.003,  0.004,  0.005,  0.006,  0.007,
        0.008,  0.009,  0.01 ,  0.011,  0.012,  0.013,  0.014,  ..., 'average']}, 'sklearn.preprocessing.MinMaxScaler': {}, 'sklearn.preprocessing.StandardScaler': {}},
        crossover_rate=0.1, cv=5, disable_update_check=False,
        early_stop=None, generations=5, max_eval_time_mins=5,
        max_time_mins=None, memory=None, mutation_rate=0.9, n_jobs=1,
        offspring_size=20, periodic_checkpoint_folder=None,
        population_size=20, random_state=None, scoring=None, subsample=1.0,
        verbosity=2, warm_start=False)

In [14]:
tpot.score(tele.drop("Class", axis=1).loc[validation_indices].values, tele.loc[validation_indices, "Class"].values)

0.87655099894847532

In [15]:
tpot.export("tpot_MAGIC_Gamma_Telescope_pipeline.py")

True